In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from models_mse import LstmMse
from models_mle import LstmMle_1, LstmMle_2, LstmMle_3
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser, PredictorMleSpecial

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [25]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/01_M02_DC_prediction_3.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION", "ROTATIONSPEED"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_data_M02Fold_xx_InputSize12_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 12,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_M02_3.csv",
    }
}

## Parameters artifical data

In [5]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/MLE_analysis/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/artifical_2_signalsFold xx_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [39]:
param = {
    "data" : {
        "path" : "./y_hat.csv",
        "droped_feature" : [
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../../models/MLE_model/degeneration_cpps_dataFold_xx_InputSize10_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "n_hidden_fc_1" : 75,
        "n_hidden_fc_2" : 25,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../../visualisation/files/prediction/MLE/cpps_degradation_yhat.csv", 
    }
}

### No standadization necessary ???

In [33]:
mean_training_data = [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]
var_training_data = [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]

## Create DataLoader

In [34]:
data_preperator = DataPreperatorPrediction(path=param["data"]["path"], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(49900, 11)


In [35]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 11])
Size of target data: torch.Size([50, 11])
Data of batch: 1
Size of input data: torch.Size([50, 100, 11])
Size of target data: torch.Size([50, 11])


## Define Model and load Parameters of trained model
### Model for MSE 

In [ ]:
model = LstmMse(batch_size=param['model']['batch_size'], 
                input_dim=param['model']['input_size'], 
                n_hidden_lstm=param['model']['n_hidden_lstm'], 
                n_hidden_fc=param['model']['n_hidden_fc_1'], 
                n_layers=param['model']['lstm_layer'], 
                dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                dropout_rate_fc= param['model']['dropout_rate_fc']
                )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

### Model for MLE

In [36]:
model = LstmMle_1(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc=param['model']['n_hidden_fc_1'], 
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [ ]:
model = LstmMle_3(batch_size=param['model']['batch_size'], 
                 input_dim=param['model']['input_size'], 
                 n_hidden_lstm=param['model']['n_hidden_lstm'], 
                 n_hidden_fc_1=param['model']['n_hidden_fc_1'], 
                 n_hidden_fc_2=param['model']['n_hidden_fc_2'],
                 n_layers=param['model']['lstm_layer'], 
                 dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                 dropout_rate_fc= param['model']['dropout_rate_fc'],
                 K = param['model']['K'])

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

## Initialize Predictor
### Predictor for MSE Model

In [ ]:
predictor = PredictorMse(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                         )

### Predictor for MLE Model

In [37]:
predictor = PredictorMle(model=model,
                         path_data=param["data"]["path"],
                         columns_to_ignore=param["data"]["droped_feature"],
                         threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                         no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                         )

In [14]:
predictor = PredictorMleSpecial(model=model,
                                path_data=param["data"]["path"],
                                columns_to_ignore=param["data"]["droped_feature"],
                                threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                                no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"],
                                deep=False, 
                                seperate=False,
                                )

## Predict

In [40]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
Current status: 15000 samples are predicted.
Current status: 20000 samples are predicted.
Current status: 25000 samples are predicted.
Current status: 30000 samples are predicted.
Current status: 35000 samples are predicted.
Current status: 40000 samples are predicted.
Current status: 45000 samples are predicted.
End of prediction.


## Tag anomalous samples

In [41]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [42]:
results.head()

,ID,signal_1 mu predicted target,signal_2 mu predicted target,signal_3 mu predicted target,signal_4 mu predicted target,signal_5 mu predicted target,signal_6 mu predicted target,signal_7 mu predicted target,signal_8 mu predicted target,signal_9 mu predicted target,...,Anomaly Sensor_1,Anomaly Sensor_2,Anomaly Sensor_3,Anomaly Sensor_4,Anomaly Sensor_5,Anomaly Sensor_6,Anomaly Sensor_7,Anomaly Sensor_8,Anomaly Sensor_9,Anomaly Sensor_10
0,201.0,0.563277,-0.364166,0.926843,0.479989,0.207464,-0.175565,0.560899,1.050689,1.571999,...,0,0,0,0,0,0,0,0,0,0
1,202.0,0.684131,-0.343719,0.306257,0.451412,0.438044,0.152068,1.026261,0.975409,1.808280,...,0,0,0,0,0,0,0,0,0,0
2,203.0,0.982035,-0.033199,-0.094953,0.631629,0.825975,0.591525,1.514147,0.808674,1.904581,...,0,0,0,0,0,0,0,0,0,0
3,204.0,1.355476,0.506549,0.156399,1.028217,1.195134,0.911548,1.684658,0.638956,1.720180,...,0,0,0,0,0,0,0,0,0,0
4,205.0,1.559834,0.871342,0.609129,1.318268,1.350047,0.993458,1.584086,0.545100,1.479633,...,0,0,0,0,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [43]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [44]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)